Task 4: Агрегации

In [1]:
import pandas as pd

In [2]:
from sqlite3 import connect

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Создайте соединение с базой данных с помощью библиотеки sqlite3.

In [4]:
con = connect('/content/drive/MyDrive/School21/day11/data/checking-logs.sqlite')

Получите схему таблицы test.

In [5]:
pd.io.sql.read_sql("PRAGMA table_info(test)",con)

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


Получите только первые 10 строк таблицы test, чтобы проверить, как она выглядит.

In [6]:
pd.io.sql.read_sql("SELECT* FROM test limit 10" ,con)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


In [7]:
pd.io.sql.read_sql("SELECT* FROM deadlines limit 10" ,con)

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


Найдите среди всех пользователей минимальное значение этой самой дельты (периода времени от даты первого коммита пользователя до срока сдачи соответствующего лабораторного задания), используя только один запрос.
Для этого сджойните данные своей таблицы с данными таблицы deadlines
Период времени должен отображаться в часах.
Не учитывайте лабораторное задание project1 с более длительным сроком сдачи, поскольку оно будет выделяться на фоне других.
Сохраните значение в датафрейме df_min с соответствующим uid.

In [8]:
df_min=pd.io.sql.read_sql('''
SELECT c.uid,
                      MIN((strftime("%s", c.first_commit_ts)-p.deadlines) / 3600) as MIN_diff
                          from test c  join deadlines p on c.labname	= p.labs
                          where p.labs not in ('project1')
                            

''',con)                
df_min

,uid,MIN_diff
0,user_30,-202


Выполните те же самые операции для максимального значения дельты, используя только один запрос. Название итогового датафрейма — df_max.

In [10]:
df_max=pd.io.sql.read_sql('''
SELECT c.uid,
                      MAX((strftime("%s", c.first_commit_ts)-p.deadlines) / 3600) as MAX_diff
                          from test c  join deadlines p on c.labname	= p.labs
                          where p.labs not in ('project1')
                            

''',con)                
df_max

,uid,MAX_diff
0,user_25,-2


,uid,MAX_diff
0,user_25,-2


Выполните те же самые операции для среднего значения дельты, используя только один запрос. На этот раз ваш итоговый датафрейм не должен включать столбец uid; имя датафрейма — df_avg.

In [11]:
df_avg=pd.io.sql.read_sql('''
SELECT 
                      AVG((strftime("%s", c.first_commit_ts)-p.deadlines) / 3600) as AVG_diff
                          from test c  join deadlines p on c.labname	= p.labs
                          where p.labs not in ('project1')
                            

''',con)                
df_avg


,AVG_diff
0,-89.125


Мы хотим проверить предположение о том, что у пользователей, посетивших ленту новостей всего несколько раз, меньше период времени между датой первого коммита и сроком сдачи лабораторного задания. Для этого вам необходимо рассчитать коэффициент корреляции между количеством просмотров страниц и разницей между датами.
Используя только один запрос, создайте таблицу со столбцами: uid, avg_diff, pageviews.
uid — это uid, существующие в таблице test.
avg_diff — среднее значение дельты (периода времени между датой первого коммита и сроком сдачи лабораторного задания) для каждого пользователя.
pageviews — количество посещений ленты новостей одним пользователем.
Не учитывайте лабораторное задание project1.
Сохраните результаты в датафрейме views_diff.
Используйте метод corr() библиотеки Pandas для вычисления коэффициента корреляции между количеством просмотров и значением дельты.

In [12]:
views_diff=pd.io.sql.read_sql('''
select t.uid,pageviews,
(strftime("%s", first_commit_ts)-deadlines) / 60/60 as AVG_diff
from 
(select uid,labname,min(first_commit_ts) as first_commit_ts 
from test 
where labname not in ('project1')
group by uid) t
  left join deadlines 
  on labname=labs
 left join
                         (SELECT uid, 
                                COUNT() as pageviews
                      FROM pageviews
                        GROUP BY uid) as vpn
on t.uid=vpn.uid
where labs not in ('project1')
group by t.uid

                         
                            

''',con)                
views_diff

,uid,pageviews,AVG_diff
0,user_1,28,-6
1,user_10,89,-39
2,user_14,143,-200
3,user_17,47,-81
4,user_18,3,-4
5,user_19,16,-148
6,user_21,10,-126
7,user_25,179,-148
8,user_28,149,-98
9,user_3,317,-75


In [13]:
views_diff.corr()

,pageviews,AVG_diff
pageviews,1.000000,-0.062967
AVG_diff,-0.062967,1.000000


Закройте соединение.

In [14]:
con.close()